In [1]:
pip install mkl-static mkl-include

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 204.8/204.8 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 15.0 MB/s eta 0:00:00


In [2]:
%pip install cmake ninja

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 422.8/422.8 kB 28.2 MB/s eta 0:00:00


In [3]:
pip install pymp-pypi

  Preparing metadata (setup.py) ... done
  Created wheel for pymp-pypi: filename=pymp_pypi-0.5.0-py3-none-any.whl size=10314 sha256=eb4ea865935d02ba1e8750ebee19c418aa078e0faf1c6bd799f0e23402196be6
  Stored in directory: /root/.cache/pip/wheels/3d/6c/ef/356575a818508a9c37c4513a5139063d522ea002bcebd15bbe
Successfully built pymp-pypi


In [4]:
pip install pymodis

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.9/58.9 kB 5.3 MB/s eta 0:00:00


In [5]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [6]:
path = "/content/drive/MyDrive/MLA/Project/images"

In [7]:
path = "/content/drive/MyDrive/Project/images"

In [10]:
import os
import requests
from datetime import datetime, timedelta
from bs4 import BeautifulSoup

# 🔧 Paramètres généraux
products = ["MOD09GQ"]  # Liste des produits
tile = "h18v04"
base_url_template = "https://e4ftl01.cr.usgs.gov/MOLT/{product}.061"
username = "m22ammad"
password = "JMles5fruits."
dest = path
os.makedirs(dest, exist_ok=True)

# 📅 Générer toutes les dates
start_date = datetime(2019, 7, 16)
end_date = datetime(2019, 12, 31)
delta = timedelta(days=1)
dates = []
current_date = start_date
while current_date <= end_date:
    dates.append(current_date.strftime("%Y.%m.%d"))
    current_date += delta

# Fichier pour loguer les dates vides
empty_dates_file = "/content/dates_vides.txt"

# 📥 Télécharger les fichiers pour chaque produit et chaque date
for product in products:
    print(f"📦 Début du téléchargement pour le produit {product}...")
    base_url = base_url_template.format(product=product)

    for date in dates:
        print(f"📆 Traitement de la date : {date} pour le produit {product}")
        day_url = f"{base_url}/{date}/"
        try:
            response = requests.get(day_url, auth=(username, password), timeout=10)
            if response.status_code != 200:
                print(f"❌ Échec d'accès à {day_url} (code {response.status_code})")
                continue

            soup = BeautifulSoup(response.text, "html.parser")
            links = soup.find_all("a")

            # 🎯 Filtrer pour les fichiers HDF de la tuile souhaitée
            hdf_links = [
                day_url + link.get("href")
                for link in links
                if link.get("href") and link.get("href").endswith(".hdf") and tile in link.get("href")
            ]

            if not hdf_links:
                print(f"⚠️ Aucun fichier HDF trouvé pour cette date et produit.")
                # Log de la date vide dans le fichier
                with open(empty_dates_file, "a") as f:
                    f.write(f"{date} - {product}\n")
                continue

            for file_url in hdf_links:
                filename = file_url.split("/")[-1]
                dest_path = os.path.join(dest, filename)
                if os.path.exists(dest_path):
                    print(f"✔️ Déjà téléchargé : {filename}")
                    continue
                print(f"⬇️ Téléchargement : {filename}")
                with requests.get(file_url, auth=(username, password), stream=True) as r:
                    with open(dest_path, 'wb') as f:
                        for chunk in r.iter_content(chunk_size=8192):
                            f.write(chunk)

        except Exception as e:
            print(f"❗ Erreur pour la date {date} et le produit {product} : {e}")

print("✅ Téléchargement terminé pour toute la période.")
print(f"Les dates sans fichiers ont été loggées dans : {empty_dates_file}")

📦 Début du téléchargement pour le produit MOD09GQ...
📆 Traitement de la date : 2019.07.16 pour le produit MOD09GQ
❗ Erreur pour la date 2019.07.16 et le produit MOD09GQ : HTTPSConnectionPool(host='e4ftl01.cr.usgs.gov', port=443): Max retries exceeded with url: /MOLT/MOD09GQ.061/2019.07.16/ (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7b8dac614590>: Failed to establish a new connection: [Errno 111] Connection refused'))
📆 Traitement de la date : 2019.07.17 pour le produit MOD09GQ
❗ Erreur pour la date 2019.07.17 et le produit MOD09GQ : HTTPSConnectionPool(host='e4ftl01.cr.usgs.gov', port=443): Max retries exceeded with url: /MOLT/MOD09GQ.061/2019.07.17/ (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7b8dbc0deb10>: Failed to establish a new connection: [Errno 111] Connection refused'))
📆 Traitement de la date : 2019.07.18 pour le produit MOD09GQ
❗ Erreur pour la date 2019.07.18 et le produit MOD09GQ : HTTPSConnectionPool(h

KeyboardInterrupt: 

In [ ]:
def date_into_n_chunk(date_list, n_chunks):
    """
    Splits a list of dates into n_chunks approximately equal-sized chunks.
    Returns a list of [start, end] pairs for each chunk.
    """
    k, m = divmod(len(date_list), n_chunks)
    chunks = [date_list[i*k + min(i, m):(i+1)*k + min(i+1, m)] for i in range(n_chunks)]
    return [[chunk[0], chunk[-1]] for chunk in chunks if chunk]  # start and end for each chunk


In [ ]:
from math import ceil
def date_into_n_chunk(l_days, n):
    """
    Description
    -----------
    Splitting the date list into n part in order to download the MODIS data
    in a parallelized fashion.

    """
    lst = [i for i in range(len(l_days))]
    size = ceil(len(lst) / n)
    l = list(map(lambda x: lst[x * size:x * size + size], list(range(n))))
    return [[l_days[k[0]],l_days[k[-1]]] for k in l]

In [ ]:
from pymodis import downmodis

import os
#import utils as us

from argparse import ArgumentParser
import pandas
from datetime import timedelta, datetime
import pymp

In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Description
-----------
Script used for downloading the modis files for the products MOD11A1 and MOD09GQ
over a time period. The products of interest could be easily modified by changing
the short_names variable inside the main.

Example
-------
python download_modis_images.py --username projet3a --password Projet3AIMT --start_date 2023-08-01 --stop_date 2023-08-08 --n_threads = 8

Else
----
@author: Romuald Ait Bachir
"""

from pymodis import downmodis

import os
#import utils as us

from argparse import ArgumentParser
import pandas
from datetime import timedelta, datetime
import pymp


def get_hdf(username: str,
            password: str,
            tiles: str,
            start: str,
            stop: str,
            short_name: str,
            n_threads: int):
    """
    Description
    -----------
    Downloading the data (hdf, xml) at tile for the instrument short_name at
    time period [start, stop].

    Let N be the number of days inside your time period. This function should
    return 2*N + 1 files (N .hidf, N .xml and 1 txt).

    The .hdf file contains the data associated to the measurement of the sensor.
    The .xml file contains the metadata of the granule.

    Parameters
    ----------
    username: string
        Username to connect to the modis session.
    password: string
        Password to connect to the modis session.
    tiles: string
        Identifier of a tile. Format: "hHHvVV" with: HH: 00->35 and VV: 00->17.
        Look at the MODIS sinusoidal tile grid for more information about a region.
    start: string
        Starting date. Format: "YYYY-MM-DD".
    stop: string
        Ending date. Format: "YYYY-MM-DD".
    short_name: string
        Shortened name referencing a product used in modis. For example: MOD11A1.061
        See the MODIS Naming Conventions at https://lpdaac.usgs.gov/data/get-started-data/collection-overview/missions/modis-overview/ for more information.
    n_threads: int
        Number of threads on which to parallelize the download.

    Example
    -------
    >>> get_hdf("johnsmith","123456789","h12v04", "2020-01-01", "2020-02-01", "MOD11A1.061", 8)

    """

    # Making the tree structure.
    #path = os.path.join('data','hdf_files',short_name)
    os.listdir("/content/drive/MyDrive/MLA/Project/images")

    # Getting the dates for the time period and splitting it into n_threads chunks for adequate multi-threading.
    l_days = [t.strftime('%Y-%m-%d') for t in list(pandas.date_range(datetime.strptime(start, '%Y-%m-%d'),datetime.strptime(stop, '%Y-%m-%d')-timedelta(days=0),freq='d'))]

    # Need to split the work as much as possible. l_days is the limit at which
    # it wouldn't be possible to fully parallelize the work
    if n_threads >= len(l_days):
        n_threads = len(l_days)

    # Processing the dates to split them into n_threads chunk for parallelization.
    l_days1 = date_into_n_chunk(l_days, n_threads)

    # Parallelizing using pymp
    with pymp.Parallel(n_threads) as p:
           for split in p.range(0,n_threads):
               start = l_days1[split][0]
               stop = l_days1[split][1]

               try:
                   print("Downloading for day {} to day {}".format(start,stop))
                   modisDown = downmodis.downModis(user=username,password=password,product=name, destinationFolder = path, tiles = tiles, today=start, enddate=stop)
                   modisDown.connect() # Connection to the server
                   modisDown.downloadsAllDay() # Download the data for the time period

               except:
                   print("Download Error, the data from tiles {} at date {} -> {} can't be downloaded".format(tiles ,start, stop))


if __name__ == "__main__":
    parser = ArgumentParser()
    # username and password should be removed if ever put somewhere online
    parser.add_argument('--username', type=str, default="projet3a")
    parser.add_argument('--password', type=str, default="Projet3AIMT")
    parser.add_argument('--start_date', type=str, default="2017-01-01")
    parser.add_argument('--stop_date', type=str, default="2019-12-31")
    parser.add_argument('--n_threads', type=int, default="12")
    # Default set to be small in order to have a simple and fast example.

    args, _ = parser.parse_known_args()

    # Parameters
    user = args.username
    passw = args.password
    debut = args.start_date
    fin = args.stop_date
    n_threads = args.n_threads

    # Products names
    short_names = ["MOD09GQ.061", "MOD21A1D.061"]
    # Note: MOD44W.061 = 1 file/year so super fast to download.
    # Much slower for MOD11A1.061 and MOD21A1D.061
    # And even more slow for MOD09GQ.061 and MOD09GA.061
    # If you want the minimal working data, only download MOD21A1D.061 and
    # MOD09GQ.061
    # !!! I also download MOD09GA RGB but don't process those later.

    # Tile of interest
    tiles = "h18v04" # centered on France, Italy, Spain, Germany

    # The download of the data is parallelized to accelerate the process
    for name in short_names:
        print("Downloading the Data of {}".format(name))
        get_hdf(username = user, password = passw, tiles = tiles, start = debut, stop = fin, short_name = name, n_threads = n_threads)

In [ ]:
# mypy: allow-untyped-defs
import collections
from itertools import repeat
from typing import Any


__all__ = ["consume_prefix_in_state_dict_if_present"]


def _ntuple(n, name="parse"):
    def parse(x):
        if isinstance(x, collections.abc.Iterable):
            return tuple(x)
        return tuple(repeat(x, n))

    parse.__name__ = name
    return parse


_single = _ntuple(1, "_single")
_pair = _ntuple(2, "_pair")
_triple = _ntuple(3, "_triple")
_quadruple = _ntuple(4, "_quadruple")


def _reverse_repeat_tuple(t, n):
    r"""Reverse the order of `t` and repeat each element for `n` times.

    This can be used to translate padding arg used by Conv and Pooling modules
    to the ones used by `F.pad`.
    """
    return tuple(x for x in reversed(t) for _ in range(n))


def _list_with_default(out_size: list[int], defaults: list[int]) -> list[int]:
    import torch

    if isinstance(out_size, (int, torch.SymInt)):
        return out_size
    if len(defaults) <= len(out_size):
        raise ValueError(f"Input dimension should be at least {len(out_size) + 1}")
    return [
        v if v is not None else d for v, d in zip(out_size, defaults[-len(out_size) :])
    ]


def consume_prefix_in_state_dict_if_present(
    state_dict: dict[str, Any],
    prefix: str,
) -> None:
    r"""Strip the prefix in state_dict in place, if any.

    .. note::
        Given a `state_dict` from a DP/DDP model, a local model can load it by applying
        `consume_prefix_in_state_dict_if_present(state_dict, "module.")` before calling
        :meth:`torch.nn.Module.load_state_dict`.

    Args:
        state_dict (OrderedDict): a state-dict to be loaded to the model.
        prefix (str): prefix.
    """
    keys = list(state_dict.keys())
    for key in keys:
        if key.startswith(prefix):
            newkey = key[len(prefix) :]
            state_dict[newkey] = state_dict.pop(key)

    # also strip the prefix in metadata if any.
    if hasattr(state_dict, "_metadata"):
        keys = list(state_dict._metadata.keys())
        for key in keys:
            # for the metadata dict, the key can be:
            # '': for the DDP module, which we want to remove.
            # 'module': for the actual model.
            # 'module.xx.xx': for the rest.
            if len(key) == 0:
                continue
            # handling both, 'module' case and  'module.' cases
            if key == prefix.replace(".", "") or key.startswith(prefix):
                newkey = key[len(prefix) :]
                state_dict._metadata[newkey] = state_dict._metadata.pop(key)

blabla

In [ ]:
%pip install numpy xarray rasterio scikit-image h5py


In [ ]:
%pip install numpy scikit-image xarray

In [ ]:
%pip install gdal-bin python3-gdal

In [ ]:
from osgeo import gdal

def read_modis_var_from_hdf_gdal(filepath, var_name):
    dataset = gdal.Open(filepath)
    if dataset is None:
        raise ValueError(f"Impossible d'ouvrir {filepath}")

    subdatasets = dataset.GetSubDatasets()
    for name, desc in subdatasets:
        if var_name in name or var_name in desc:
            ds = gdal.Open(name)
            array = ds.ReadAsArray()
            return array
    raise ValueError(f"Variable {var_name} non trouvée dans {filepath}")

In [ ]:
read_modis_var_from_hdf_gdal("/content/drive/MyDrive/Project/images/MOD09GQ.A2017012.h18v04.061.2021250221609.hdf","sur_refl_b01")

In [ ]:
from osgeo import gdal

# Ouvre le fichier HDF
hdf_path = "/content/drive/MyDrive/MLA/Project/images/MOD09GQ.A2017012.h18v04.061.2021250221609.hdf"
hdf_file = gdal.Open(hdf_path)

# Affiche les sous-datasets disponibles
subdatasets = hdf_file.GetSubDatasets()
for i, sds in enumerate(subdatasets):
    print(f"{i}: {sds[0]}")

# Exemple : lire le premier sous-dataset (Band 1)
sds_path = subdatasets[0][0]
band1 = gdal.Open(sds_path)
array = band1.ReadAsArray()

print(array.shape)

Bloc pour interagir avec le fichier .hdf.xml ⬇️

In [ ]:
hdf_xml_path = "/content/drive/MyDrive/MLA/Project/images/MOD09GQ.A2017012.h18v04.061.2021250221609.hdf.xml"
